# Combine Movie Functions

In [4]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from wikipedia import wikipedia
from wikifilm import wikifilm
from oscar import oscars
from razzies import razzies
from rollingstone import rollingstone
from BAFTA import BAFTA
from SAG import SAG
from filmsite import filmsite
from films101 import films101
from AACTA import AACTA
from flops import flops
from canada import Canada
from boxofficemojo import boxofficemojo
from rottentomatoes import rottentomatoes
from ultimatemovierankings import ultimatemovierankings
from combine import combine


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-03-02 11:46:00.583184


# Get/Parse/Merge/Process Rotten Tomatoes Data

In [31]:
%load_ext autoreload
%autoreload

comb = combine()
comb.setOscarData(oscars(wikipedia()), 10)
comb.setWikiFilmData(wikifilm(), 1)
comb.setRazziesData(razzies(), 1)
comb.setBAFTAData(BAFTA(), 1)
comb.setSAGData(SAG(), 1)
comb.setAACTAData(AACTA(), 1)
comb.setFilmsiteData(filmsite(), 1)
comb.setFilms101Data(films101(), 1)
comb.setFlopsData(flops(), 1)
comb.setCanadaData(Canada(), 1)
comb.setRollingStoneData(rollingstone(), 1)
comb.setRottenTomatoesData(rottentomatoes(), 90)
comb.setBoxOfficeMojoData(boxofficemojo(), 5e6)
comb.setUltimateMovieRankingsData(ultimatemovierankings(), 90)
comb.getData()
comb.saveCorrections(debug=True)
comb.mergeMovies(debug=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Found 67 Years of Ultimate Movie Rankings Movies
Found 119 Years of Films101 Movies
Found 61 Years of Canada Movies
Found 105 Years of Filmsite Movies
Found 98 Years of Wiki Film Movies
Found 47 Years of BAFTA Movies
Found 94 Years of Rotten Tomatoes Movies
Found 23 Years of Rolling Stone Movies
Found 25 Years of SAG Movies
Found 39 Years of Box Office Mojo Movies
Found 38 Years of Razzies Movies
Found 50 Years of AACTA Movies
Found 84 Years of Oscar Movies
Found 32 Years of Flops Movies
Found Data Between 1900 and 2018
Found 820 old corrections
Found 35 new corrections
Adding 28 Days Later...
Adding new corrections to Au Revoir Les Enfants
Adding D.O.A.
Adding Disneyland: I Captured The King Of The Leprechauns
Adding new corrections to Dracula
Adding Drácula
Adding Dumb And Dumber
Adding Fail-Safe
Adding new corrections to Fanny And Alexander
Adding Good Bye Lenin!
Adding Halloween H20: 20 Years La

# Find Corrections

In [2]:
from ioUtils import getFile
data = getFile("/Users/tgadfort/Documents/code/movies/combine/results/movies.json")
for movie,name in data.items():
    year  = movie[-5:-1]
    movie = movie[:-7]
    
    pos = movie.rfind("(")
    if pos == -1:
        continue        

    if pos > len(movie) - 15 and movie.find(",") != -1:
        fix = movie[:pos].strip()
        print("{0}:".format(fix))
        print(" - {0}".format(movie))

Walt Disney Treasures - The Chronological Donald, Volume One:
 - Walt Disney Treasures - The Chronological Donald, Volume One (1934 - 1941)
Walt Disney Treasures - The Chronological Donald, Volume Three:
 - Walt Disney Treasures - The Chronological Donald, Volume Three (1947 - 1950)
Aparajito:
 - Aparajito (1956, India)
Walt Disney Treasures - The Chronological Donald, Volume Four:
 - Walt Disney Treasures - The Chronological Donald, Volume Four (1951 - 1961)


In [28]:
from searchUtils import findNearest
from fsUtils import setFile
from ioUtils import saveFile

savename = setFile(comb.getDataDir(), "saved.yaml")
savedData = getFile(savename)

movies = comb.getCombinedMovies()

repData = {}
for corrMovie,corrs in savedData.items():
    for corr in corrs:
        try:
            repData[corr] = corrMovie
        except:
            raise ValueError(corr,corrMovie)
            
print("Found {0} movies".format(len(movies)))
print("Found {0} corrections".format(len(repData)))

Found 16533 movies
Found 1010 corrections


In [29]:
movielist = {}
for moviename in movies.keys():
    name = moviename[:-7]
    year = moviename[-5:-1]
    try:
        year = int(year)
    except:
        raise ValueError(moviename)

    if repData.get(name) is not None:
        name = repData[name]

    if name.find("IMAX") != -1:
        continue
    if name.find("re-issue") != -1:
        continue        
        
    for y in range(year-2,year+3):
        if movielist.get(y) is None:
            movielist[y] = []
        movielist[y].append(name)

for year in movielist.keys():
    movielist[year] = set(movielist[year])

In [30]:
data   = {}
cutoff = 0.85

for year,movienames in movielist.items():
    print(year,len(movienames))
    
    for im,movie in enumerate(list(movienames)):
        if movie.endswith(' '):        
            if data.get(movie[:-1]) is None:
                print('\t',len(data),'\t',movie[:-1])
                data[movie[:-1]] = set()
                data[movie[:-1]].add(movie)
            tmpList = list(filter(lambda a: a != movie[:-1], list(movienames)))
            results = findNearest(movie[:-1], tmpList, 3, cutoff)
            if len(results) > 0:
                for value in results:
                    data[movie[:-1]].add(value)
        else:
            tmpList = list(filter(lambda a: a != movie, list(movienames)))    
            results = findNearest(movie, tmpList, 3, cutoff)
            if len(results) > 0:
                if data.get(movie) is None:
                    print('\t',len(data),'\t',movie)
                    data[movie] = set()
                for value in results:
                    data[movie].add(value)

        if len(data) > 300:
            break
        
for k in data.keys():
    data[k] = list(data[k])

            

savename = setFile(comb.getDataDir(), "corr.yaml")
saveFile(idata=data, ifile=savename, debug=True)

1898 8
1899 19
1900 25
1901 33
1902 42
1903 41
1904 38
1905 42
1906 41
1907 45
1908 64
1909 75
1910 93
1911 109
1912 130
1913 129
1914 153
1915 159
1916 165
1917 163
1918 181
1919 179
	 0 	 The Boat
	 1 	 The Goat
1920 196
1921 209
	 2 	 The Spoilers
	 3 	 The Spiders
1922 220
1923 235
1924 258
	 4 	 What Price Goofy?
	 5 	 What Price Glory?
1925 275
1926 314
1927 367
1928 379
	 6 	 Up The River
	 7 	 The River
1929 398
	 8 	 Dracula
	 9 	 Drácula
1930 417
1931 441
1932 434
1933 460
	 10 	 Gold Diggers Of 1935
	 11 	 Gold Diggers Of 1933
1934 476
	 12 	 Garden Of Allah
	 13 	 The Garden Of Allah
	 14 	 Rose Marie
	 15 	 Rose-Marie
1935 481
	 16 	 Charlie Chan At The Circus
	 17 	 Secret Agent
	 18 	 Secret Agent X-9
	 19 	 Charlie Chan At The Olympics
1936 471
	 20 	 Peter The First, Part One
	 21 	 Peter The First, Part Two
1937 476
1938 473
	 22 	 Popeye The Sailor: 1938-1940, Volume 2
	 23 	 Popeye The Sailor: 1933-1938, Volume 1
1939 480
	 24 	 Tom, Dick And Harry
	 25 	 Tom, Dick,

	 287 	 Cheaper By The Dozen
	 288 	 Cheaper By The Dozen 2
2004 1552
	 289 	 Scary Movie 4
	 290 	 Final Destination 3
	 291 	 Deliver Us From Evil
	 292 	 Saw III
	 293 	 Deliver Us From Eva
	 294 	 The Grudge
	 295 	 The Grudge 2
2005 1570
	 296 	 King Of Kong A Fistful Of Quarters
	 297 	 Alexander
	 298 	 Saw IV
	 299 	 The Cookout
	 300 	 Alexandra
2006 1605
2007 1595
2008 1580
2009 1580
2010 1570
2011 1546
2012 1504
2013 1512
2014 1508
2015 1496
2016 1454
2017 1165
2018 868
2019 545
2020 252
Saving data to /Users/tgadfort/Documents/code/movies/combine/data/corr.yaml
Saved data to /Users/tgadfort/Documents/code/movies/combine/data/corr.yaml
  --> This file is 12.7kB.
